In [1]:
from dotenv import load_dotenv
load_dotenv('azure.env')
import os

In [2]:
from lyzr_automata.ai_models.openai import AzureOpenAIModel

# We will first create open ai model for our language tasks
# and set params according to chat completion.

open_ai_model_text = AzureOpenAIModel(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
      api_version="2024-02-15-preview",
    parameters={
        "model": os.environ.get('AZURE_OPENAI_GPT_DEPLOYMENT'),
        "temperature": 0.2,
        "max_tokens": 1500,
    },
)

In [3]:
open_ai_model_image = AzureOpenAIModel(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
      api_version="2024-02-15-preview",
    parameters={
        "n": 1,
        "model": "dall-e-3",
    },
)

In [4]:
from lyzr_automata import Agent

# Content researcher agent for our search task
content_researcher_agent = Agent(
    prompt_persona="You are an AI journalist good at using the provided data and write an engaging article",
    role="AI Journalist",
)

# Linkedin content creator agent for your linkedin content writing task
linkedin_content_writer_agent = Agent(
    prompt_persona="You write engaging linkedin posts with the provided input data",
    role="Linkedin Content Creator",
)

In [5]:
from typing import Optional

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.tools import BaseTool

from langchain_community.utilities.bing_search import BingSearchAPIWrapper

def search_internet(search_query):
    """
    Tool that queries the Bing Search API and gets back json.
    
    Parameters:
    - query (str): Search Query.
    
    Returns:
    - str: Search Results
    """
    api_wrapper = BingSearchAPIWrapper()
    return str(api_wrapper.results(search_query, 20))

#INPUT
from pydantic import BaseModel

class SearchInternetInput(BaseModel):
    search_query: str  

#OUPUT
class SearchInternetOutput(BaseModel):
    result: str

from lyzr_automata import Tool

search_internet_tool = Tool(
        name="Search Internet tool",
        desc="Tool that queries the Bing Search API and gets back json",
        function=search_internet,
        function_input=SearchInternetInput,
        function_output=SearchInternetOutput,
        default_params = {}
    )

In [6]:
from lyzr_automata.tasks.task_literals import InputType, OutputType
from lyzr_automata import Task


search_task = Task(
    name="Search Latest AI News",
    output_type=OutputType.TEXT,
    input_type=InputType.TEXT,
    model=open_ai_model_text,
    instructions="Search and collect all latest news about the topic 'Autogen from Microsoft'",
    log_output=True,
    tool=search_internet_tool
)

research_task = Task(
    name="Draft Content Creator",
    agent=content_researcher_agent,
    output_type=OutputType.TEXT,
    input_type=InputType.TEXT,
    model=open_ai_model_text,
    instructions="Analyze the input and clean the data and write content which can be used to create medium blog in the next task",
    log_output=True,
    enhance_prompt=False,
)

linkedin_content_writing_task = Task(
    name="Linkedin Post Creator",
    agent=linkedin_content_writer_agent,
    output_type=OutputType.TEXT,
    input_type=InputType.TEXT,
    model=open_ai_model_text,
    instructions="Use the news summary provided and write 1 engaging medium blog",
    log_output=True,
    enhance_prompt=False,

)
image_creation_task = Task(
    name="Linkedin Image Creation",
    output_type=OutputType.IMAGE,
    input_type=InputType.TEXT,
    model=open_ai_model_image,
    log_output=True,
    instructions="Use the research material provided and create a medium blog image that would be suitable for posting",
)

In [7]:
# Use our LinearSyncPipeline to run tasks linearly in sync way
# We are also using summarize_task from our prebuilt tasks because image creation dalle model has a shorter limit
from lyzr_automata.tasks.util_tasks import summarize_task
from lyzr_automata.pipelines.linear_sync_pipeline import LinearSyncPipeline

def main():
    LinearSyncPipeline(
        name="Automated medium blog creation pipeline",
        completion_message="Wrote the Post successfully",
        tasks=[
            search_task,
            research_task,
            linkedin_content_writing_task,
            image_creation_task
        ],
    ).run()


main()

START PIPELINE Automated medium blog creation pipeline :: start time : 1713157050.0483258
START TASK Search Latest AI News :: start time : 1713157050.0488424
output : [{'snippet': 'AutoGenBench is a commandline tool for downloading, configuring, running an agentic benchmark, and reporting results. It is designed to allow repetition, isolation and instrumentation, leveraging the new runtime logging feature. These tools have been used for improving the <b>AutoGen</b> library as well as applications.', 'title': 'What&#39;s New in AutoGen? | AutoGen - microsoft.github.io', 'link': 'https://microsoft.github.io/autogen/blog/2024/03/03/AutoGen-Update/'}, {'snippet': 'Chi Wang discusses the <b>latest</b> updates on <b>AutoGen</b>—the multi-agent framework for next-generation AI applications. This includes milestones achieved, community feedback, new exciting features, and ongoing research and challenges. <b>Microsoft</b> Research Forum, March 5, 2024. CHI WANG: Hi, everyone. My name is Chi.', 